In [11]:
class attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
#         hidden = [batch_size, dec_hid_dim]
#         encoder_outputs = [src_len, batch_size, enc_hid_dim*2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
#         repeat decoder hidden state src_len times
        hidden = hidden.unsqeeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.pertute(1, 0, 2)
        
#         hidden = [batch size, src len, dec hid dim]
#         encoder_outputs = [batch size, src len, enc hid dim * 2]
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [12]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()
        
        self.out_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2] 2를 곱해준 이유는 .. backward를 고려한 것이 아닐까 생각해봄 
        input = input.unsqeeze(0)
        # 훈련할 때 
        #input = [1, batch size]
        #여기서 인풋은 tgt이겠지?
        #embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input)) 
        # 인코더 히든 차원, 디코더히든 차원
        a = self.attention(hidden, encoder_outputs)
        # a는 소프트 맥스까지 통과한 확률값이 나온다..??
        a = a.unsqeeze(1)
        # a = [batch size, src len]
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        # encoder_outputs = [batch size, src len, enc hid dim * 2]
        weighted = torch.bmm(a, encoder_outputs)
        # 가중치를 인코더의 히든 층에 곱한 가중합??
        #weighted = [batch size, 1, enc hid dim * 2]
        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim * 2]
        # 인베딩도니 벡터와 맞춰주는 건가?
        # 바우나우 어텐션은 임베딩 차원과 가중합된 context vector를 concat하여 사용
        rnn_input = torch.cat((embedded, weighted), dim=2)
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
        output, hidden = self.rnn(rnn_input, hidden.unsqeeze(0))
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        # linear 함수를 통해 값을 represent한다.
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)